# Adding and updating documents in MongoDB

In this module, you'll practice creating NoSQL documents, learn more about document structures (embedded documents, arrays, and arrays of embedded documents), and how to update documents.


## Projects, Databases and Collections

First, let's contrast MongoDB to an RDBMS.

### Projects

Projects are a concept that MongoDB has - Projects are the server cluster that houses a set of databases. For our local MongoDB installation, we don't have a project.

### Databases

In a relational database management system, we create objects called databases. 

The concept of a database also applied to Mongo DB. Using the MongoDB Atlas cloud service, Databases exist within a Project. 

### Collections (vs Tables)

In a RDBMS database, we have a data base structure the contains tables and relationships between tables. Inside these stuctures we store rows of data. Each row typically has multiple-columns. Each columns has an associated type, and all data stored in a columns must conform to this type. 

MondgoDB stores data records as documents which are gathered together into collections. Thus, a database in MongoDB stores one or more collections of documents. The stucture of each document consists of fields assembled in a JSON like structure.

### A mongoDB document

A document is a json file. Documents have dynamic schema. Dynamic schema means that documents in the same collection do not need to have the same set of fields or structure, and common fields in a collection's documents may hold different types of data.

## Summary

The following table shows the relationship of RDBMS terminology with MongoDB.

| RDBMS  | MongoDB |
| -------| --------|
| Database | Database |
| Table | Collection |
| Tuple/Row | Document |
| column | Field |
| Table Join | Embedded Documents |
| Primary Key | Primary Key* |

\* NOTE: Default key _id provided by MongoDB itself

## Creating a database in MongoDB

### Activity 1: Create a new database and collection

Sign into your account and create a new database called Cardio. In this database we will store collections of documents pertaining cardiology patients. Inside this database, create a collection called patient_monitoring.

Once this is done, connect to your database.

In [1]:
import pymongo # pymongo is a python driver for MongoDB

client = pymongo.MongoClient('mongodb://localhost:27017/')

db = client.cardio # this is a 'database'
collection = db.patients # this is a 'collection'

# note: if you rerun this notebook, you will attempt to write the same documents into the database that already exist. 
# This will cause an error, as we will be inserting duplicate keys.
# To avoid this, you can drop the collection first:
collection.drop() # if you rerun this notebook, you'll keep addding to the data - so, drop the collection first


> NOTE: If the database doesn't exist (in this cardio), Mongodb will create it - otherwise, it will be appended

### Activity 2: Insert data into database
Insert documents into our patient_monitoring collection. We'll synthesize a number of documents using Python.

Our document data will look like the following example (note: this is the same structure as dictionaries and lists in Python - but it's not Python, it's JSON):

```json
{ 
    patient_info: {
        fname: 'Tim', 
        lname: 'Smith'
    }
    phone_numbers: [
        '555-321-456',
        '555-123-456',
        '555-222-456'
    ]
    readings: [
        {'Systolic': 125, 'Diastolic': 80, 'Rate': 70 },
        {'Systolic': 131, 'Diastolic': 85, 'Rate': 75 },
        {'Systolic': 127, 'Diastolic': 79, 'Rate': 78 },
        {'Systolic': 121, 'Diastolic': 85, 'Rate': 73 },      
    ]
}
```

Now, create 50 new patients and insert them into the collection:

In [2]:
from random import randint, seed
import json

seed(1) # make the data reproducible (which will be important for later querrying)

fnames = ['John','Betty','July', 'James','Richard','Ethan', 'Liam','Jack','Jerry','Jordan','Susan','Elizabeth', 'Tom']
lnames = ['Jones', 'Smith', 'Carmichael', 'Granger', ' Fox', 'Madden', 'Hart', 'Boone', 'Hale', 'Langston']

for id  in range(1, 51): # id will be used as the _id field in the document. 

    office_num = randint(1, 10),
    
    patient_info = {   
        'fname': fnames[randint(0,(len(fnames)-1))],
        'lname': lnames[randint(0,(len(lnames)-1))],
    }    
    
    phone_numbers = [] 
    for i in range(randint(2,4)):
        phone_number = ''
        for j in range(10):
            phone_number += str(randint(0,9))
            if j == 2 or j == 5 :
                phone_number += '-'
        phone_numbers.append(phone_number)

    weights = [randint(90, 400) for x in range(randint(1,5))]
    height = randint(100, 200)

    bp_readings = [] 
    for i  in range(randint(2,10)): 
        reading = {
            'Systolic': randint(100, 150),
            'Diastolic':  randint(70, 99),
            'Rate': randint(60, 110)
        }
        bp_readings.append(reading)

    patient_doc = { # here will build the final documnet
        '_id': id,
        'office_num': office_num,
        'patient_info': patient_info, # this is an embedded document
        'patient_numbers': phone_numbers, # this is an array in the document
        'weights': weights, # this is an array in the document
        'height': height,
        'bp_readings': bp_readings, # this is an array of embedded documents
    }
    
    result = collection.insert_one(patient_doc)   # insert the randomly created document into the collection
    


> NOTE: If patients collection doesn't exist it will be created, if it does exist, it will be appended.

## Adding data to existing documents

### Ex. 1: Adding a new field to the document

In [3]:
collection.update_one(
   { '_id': 1},
   { '$set': { 'comments': "good patient"}}  # use set when you just want to change/set a new value, push if you wish to add/append a new value to the field (which will result in creating an array)
)

In [4]:
for patient in db.patients.find({'_id': 1}):
    print(json.dumps(patient, indent=4)) # json.dumps will format the document in a pretty format

{
    "_id": 1,
    "office_num": [
        3
    ],
    "patient_info": {
        "fname": "Jordan",
        "lname": "Smith"
    },
    "patient_numbers": [
        "177-763-1706",
        "690-743-9150",
        "008-063-6083"
    ],
    "weights": [
        343,
        373,
        209,
        266
    ],
    "height": 129,
    "bp_readings": [
        {
            "Systolic": 148,
            "Diastolic": 84,
            "Rate": 78
        },
        {
            "Systolic": 101,
            "Diastolic": 83,
            "Rate": 95
        },
        {
            "Systolic": 141,
            "Diastolic": 73,
            "Rate": 71
        },
        {
            "Systolic": 140,
            "Diastolic": 93,
            "Rate": 78
        },
        {
            "Systolic": 107,
            "Diastolic": 93,
            "Rate": 81
        }
    ],
    "comments": "good patient"
}


### Ex. 2: Adding a new field to a embedded document

In [5]:
db.patients.update_one(
   { '_id': 1},
   { '$set': { 'patient_info.mname': 'Jane'}} # we're creating an setting a new field in patient_info called mname
)

In [6]:
for patient in db.patients.find({'_id': 1}):
    print(json.dumps(patient, indent=4))

{
    "_id": 1,
    "office_num": [
        3
    ],
    "patient_info": {
        "fname": "Jordan",
        "lname": "Smith",
        "mname": "Jane"
    },
    "patient_numbers": [
        "177-763-1706",
        "690-743-9150",
        "008-063-6083"
    ],
    "weights": [
        343,
        373,
        209,
        266
    ],
    "height": 129,
    "bp_readings": [
        {
            "Systolic": 148,
            "Diastolic": 84,
            "Rate": 78
        },
        {
            "Systolic": 101,
            "Diastolic": 83,
            "Rate": 95
        },
        {
            "Systolic": 141,
            "Diastolic": 73,
            "Rate": 71
        },
        {
            "Systolic": 140,
            "Diastolic": 93,
            "Rate": 78
        },
        {
            "Systolic": 107,
            "Diastolic": 93,
            "Rate": 81
        }
    ],
    "comments": "good patient"
}


### Ex. 3: Adding to array of embedded documents

We're adding a new document to the array, so we use push

In [7]:
db.patients.update_one(
   { '_id': 1},
   { '$push': { 'readings': {"Systolic": 999,"Diastolic": 999,"Rate": 999}}} 
)

In [8]:
for patient in db.patients.find({'_id': 1}):
    print(json.dumps(patient, indent=4))

{
    "_id": 1,
    "office_num": [
        3
    ],
    "patient_info": {
        "fname": "Jordan",
        "lname": "Smith",
        "mname": "Jane"
    },
    "patient_numbers": [
        "177-763-1706",
        "690-743-9150",
        "008-063-6083"
    ],
    "weights": [
        343,
        373,
        209,
        266
    ],
    "height": 129,
    "bp_readings": [
        {
            "Systolic": 148,
            "Diastolic": 84,
            "Rate": 78
        },
        {
            "Systolic": 101,
            "Diastolic": 83,
            "Rate": 95
        },
        {
            "Systolic": 141,
            "Diastolic": 73,
            "Rate": 71
        },
        {
            "Systolic": 140,
            "Diastolic": 93,
            "Rate": 78
        },
        {
            "Systolic": 107,
            "Diastolic": 93,
            "Rate": 81
        }
    ],
    "comments": "good patient",
    "readings": [
        {
            "Systolic": 999,
            "Diast

### Close Database connection

Always good to close the database connect when finished.

In [9]:
client.close()